# Introduction à l'apprentissage par renforcement
# TP 1 - les manchots multi-bras

Etudiant·e : AJOUTER VOTRE NOM ICI

1/4 de la note finale est liée à la mise en forme : 

* pensez à nettoyer les outputs inutiles (installation, messages de débuggage, ...)
* soignez vos figures : les axes sont-ils faciles à comprendre ? L'échelle est adaptée ? 
* commentez vos résultats : vous attendiez-vous à les avoir ? Est-ce étonnant ? Faites le lien avec la théorie.

Ce TP reprend l'exemple d'un médecin et de ses vaccins. Vous allez comparer plusieurs stratégies et trouver celle optimale.
Un TP se fait en groupe de 2 à 4. Aucun groupe de plus de 4 personnes. 

Vous allez rendre le TP dans une archive ZIP. L'archive ZIP contient ce notebook au format `ipynb`, mais aussi exporté en PDF & HTML. 
L'archive ZIP doit aussi contenir un fichier txt appelé `groupe.txt` sous le format:

```
Nom1, Prenom1, Email1, NumEtudiant1
Nom2, Prenom2, Email2, NumEtudiant2
Nom3, Prenom3, Email3, NumEtudiant3
Nom4, Prenom4, Email4, NumEtudiant4
```

Un script vient extraire vos réponses : ne changez pas l'ordre des cellules et soyez sûrs que les graphes sont bien présents dans la version notebook soumise. 

In [2]:
! pip install matplotlib tqdm numpy ipympl opencv-python
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu execute kernel
kernelspec migrate nbconvert nbextension 

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

import typing as t
import math
import torch
import numpy as np
from tqdm.auto import trange, tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import cv2
from IPython.display import display, clear_output

torch.random.manual_seed(0)

K = 5 # num arms

# Présentation du problème

In [4]:
class ArmBernoulli:
    def __init__(self, p: float, random_state: t.Optional[int] = None):
        """
        Vaccine treatment following a Bernoulli law (mean is p and variance is p(1-p)
        Args:
             p (float): mean parameter
             random_state (int): seed to make experiments reproducible
             
        >>> arm = ArmBernoulli(0.5, 0)
        >>> arm.sample(5)
        tensor([ True, False,  True,  True,  True])
        """
        if random_state is not None:
            torch.random.manual_seed(random_state)
        self.immunity_rate = p

    def sample(self, n: int = 1):
        return torch.rand(n) < self.immunity_rate
    
    def __repr__(self):
        return f'<ArmBernoulli p={self.immunity_rate}' 

def generate_arms(num_arms: int):
    means = torch.rand(K)
    MAB = [ArmBernoulli(m) for m in means]
    assert MAB[0].immunity_rate == means[0]
    assert (MAB[0].sample(10) <= 1).all() and (MAB[0].sample(10) >= 0).all() 
    return MAB

MAB = generate_arms(K)

**Note importante :** pour la suite, les tests seront faits avec 10 MAB différents ou plus pour réduire le bruit de simulation. 

Ce TP reprend l'exemple du médecin présenté en cours.


**Q1. Que vaut $\mu^*$ avec `random_state = 0`? Comment est définie la récompense $R_k$ ? Que représente concrètement le regret dans cet exemple ?**

In [5]:
def test_generate_10MAB(num_arms: int):
    means = torch.rand(K * 10)
    DIX_MAB = [ArmBernoulli(m, 0) for m in means]
    return DIX_MAB

print(max([arm.immunity_rate for arm in test_generate_10MAB(K)]))

tensor(0.9971)


$\mu^*$ represente simplement l'`immunity_rate` lorsque `random_state = 0`, ici fixer le `random_state` permet d'avoir une expérience reproductible. On obtient $mu^*$ = 0.9527 avec 10 MAB lors de cette expérience.

La récompense $R_k$ est définie comme la récompense obtenue pour le patient k, elle est égale a 1 s’il est immunisé, et 0 sinon.

Dans cet exemple, le regret représente concrètement  

# I. Cas classique des bandits manchots 

## I.a. Solution Gloutonne

Le médecin fonctionne sur deux phases :

1. **Exploration :** N patients reçoivent une dose d'un vaccin choisi aléatoirement. Le médecin calcule le taux d'immunisation empirique :

$$\bar{R_i} = \frac{1}{T_i} \sum_{k=0}^{N-1} \chi_{v_k,i}R_k,$$

avec $T_i = \sum_{k=0}^{N-1} \chi_{v_k,i}$.


2. **Exploitation :** Le vaccin $v_i = \arg\max_j \bar{R_j}$ est utilisé pour les M patients suivants. C'est la phase de test.

**Q2. Implémentez la solution gloutonne avec N = 50 et M = 500 et testez la avec 100 MAB différents (tous ont 5 vaccins). On s'intéresse à la variable aléatoire "la phase d'exploration a trouvé le bon vaccin". Quelle est l'espérance empirique de cette variable ? Et son écart-type ? Calculez de même l'espérance et l'écart-type du regret sur vos 100 simulations.**

Pour rappel, le regret est défini par :

$$r_n = n\mu^* - \sum_{k=0}^{n-1} R_k$$

**Attention :** $n$ est le nombre total de patients, donc ici $N + M$.

In [10]:
import random

# Solution glutonne
def get_vaccines_efficency(vaccines, N):
    r_k = [vaccine.sample() for vaccine in vaccines]
    T_i = {vaccines[i] : vaccines.count(vaccines[i]) for i in range(N - 1)}
    result = []
    for vaccine_i in vaccines:
        sum_x_rk = 0
        k = 0
        for vaccine_k in vaccines:
            X = int(vaccine_i == vaccine_k)
            sum_x_rk += X * r_k[k]
            k += 1
        result.append((1/T_i[vaccine_i]) * sum_x_rk)
    return result
    
# Regret
def get_regret(vaccines, N, nu_star):
    r_ks = sum([int(vaccine.sample()) for vaccine in vaccines])
    return N * nu_star - r_ks


# Liste de tous les vaccins injectés aux Npatients
def get_vaccines_injected(N, MAB):
    vaccines = []
    for _ in range(N):
        random_vac = random.randint(0, K - 1)
        # ADD THE VACCINE OBJECT IN LIST (tuple?)
        vaccines.append(MAB[random_vac])
    return vaccines

N = 50
M = 500

# On génére les résultats d'une injection à N patients
vaccines_train = get_vaccines_injected(N, MAB)

# On choisit le meilleur vaccin d'après le batch de test de N patient
R_i = get_vaccines_efficency(vaccines_train, N)

best_vaccine_i = R_i.index(max(R_i))
print("The best vaccine has for empirical efficency:",  best_vaccine_i, max(R_i))

# On verifie par batch si c'est le bon vaccin
vaccines_test = get_vaccines_injected(M, MAB)
R_i_test = get_vaccines_efficency(vaccines_test, M)
best_vaccine_i_test = R_i_test.index(max(R_i_test))
print("The best vaccine on TEST SET has for empirical efficency:",  best_vaccine_i_test, max(R_i_test))

# S'il est mauvais on choisit un nouveau vaccin en recalculant les R_i (N + m)

# On refait l'opération jusqu'à avoir fait tout le batch de M patients

# On calcule le regret avec la mean du meilleur vaccin trouvé sur les M patients
nu_star = max(R_i_test)
regret = get_regret(vaccines_test, M, nu_star)
print("The regret on TEST SET is:", regret)

# Implementation de l'algorithme pour la q2
def q2():
    N = 50
    M = 500

    found_exploration = []
    regrets = []

    for i in range(100):
        new_MAB = generate_arms(K)
        vaccines_train = get_vaccines_injected(N, new_MAB)
        R_i = get_vaccines_efficency(vaccines_train, N)
        best_vaccine_i = R_i.index(max(R_i))

        vaccines_test = get_vaccines_injected(M, new_MAB)
        R_i_test = get_vaccines_efficency(vaccines_test, M)
        best_vaccine_i_test = R_i_test.index(max(R_i_test))

        found_exploration.append(best_vaccine_i == best_vaccine_i_test)

        nu_star = max(R_i_test)
        regret = get_regret(vaccines_test, M, nu_star)
        regrets.append(regret.numpy()[0])
    
    print("La moyenne empirique de la VA est :", sum(found_exploration)/len(found_exploration))
    print("L'ecart type de la VA est :", np.std(found_exploration))

    print("La moyenne empirique du regret est :", sum(regrets)/len(regrets))
    print("L'ecart type du regret est :", np.std(regrets))

q2()

The best vaccine has for empirical efficency: 1 tensor([0.6667])
The best vaccine on TEST SET has for empirical efficency: 1 tensor([0.7593])
The regret on TEST SET is: tensor([204.6296])
La moyenne empirique de la VA est : 0.1
L'ecart type de la VA est : 0.30000000000000004
La moyenne empirique du regret est : 170.09489624023436
L'ecart type du regret est : 59.51234


**Q3. On propose d'améliorer l'algorithme précédant en mettant à jour les taux d'immunisation empiriques $\bar{R}_i$ pendant la d'exploration. Notez vous une amélioration du regret ? Proposez un exemple dans lequel cette mise à jour ne changera rien.** 

*[Ajoutez votre commentaire ici]*

**Q4. Créez une figure contenant deux sous-figures : à gauche, le taux d'immunisation empirique $\bar{R}_i$ pour les 5 vaccins ; à droite, le regret $r_n$. La figure sera animée avec les patients : chaque frame $k$ de l'animation représente le vaccin que l'on donne au $k$-ième patient.**

*[Ajoutez votre commentaire ici]*

**Q5. On étudie maintenant l'influence de la taille du training set $N$. On considère que N+M est une constante, puis on fait varier N entre K et M. Calculez le regret pour ces différentes tailles du training set  différents MAB et representez le regret moyen, le regret min et max (vous devriez trouver une courbe en U ou en V pour le regret moyen). Quelle est la taille optimale du training set ?**

*[Ajoutez votre commentaire ici]*

## I.b. Borne inférieure de Lai & Robbins [Lai et Robbins, 1985]

Pour un modèle de manchot de Bernoulli (équivalent au problème étudié), la borne inférieure de Lai et Robbins [Lai et Robbins, 1985] stipule que :

$$\lim \inf_{n\rightarrow \infty} \frac{\sum_{k=0}^{n-1} R_k}{\log n} \geq \sum_{i~\text{tel que}~\mu_i \lt \mu^*} \frac{\mu^∗−\mu_i}{\text{KL}(\mu_i, \mu^*)}  :=C(\mu)$$
 
 avec $\text{KL}(x, y) = x \log(x/y) + (1 − x) \log((1 − x)/(1 − y))$ (distance de Kullback-Leibler) et  $\sum_{k=0}^{n-1} R_k$ la récompense obtenue sur $n$ patients (avec un algorithme optimal). 
 
 
**Q6. Justifiez pourquoi cette borne signifie que la machine optimale est jouée exponentiellement plus souvent que les autres machines.**

*[Ajoutez votre commentaire ici]*

**Q7. Tracez le regret issu de la borne de Lai & Robbins et comparez le au regret obtenu avec l'algorithme glouton.**

*[Ajoutez votre commentaire ici]*

## I.c. Upper Confidence Bounds

Cet algorithme améliore la version précédente en ajoutant un biais lié à la fréquentation de chaque vaccin :

$$\hat{R}_i = \bar{R}_i + \sqrt{\frac{C\log{n}}{T_i}}$$,

avec $C=2$.

**Q8. Implémentez la modification de cette algorithme. Conservez les deux phases exploration/exploitation décrites ci-dessus. En prenant les valeurs de $N$ et $M$ trouvées à la question Q5, quel regret obtenez-vous ? Faites l'expérience avec au moins 10 MAB différents (tous ayant 5 vaccins) afin de calculer la moyenne et l'écart-type du regret.**

*[Ajoutez votre commentaire ici]*

**Q9. Reprenez la questions Q4 avec cette algorithme. Dans la figure de gauche, vous representerez $\bar{R}_i$ et $\hat{R}_i$.**

*[Ajoutez votre commentaire ici]*

**Q10. Reprenez la question Q5 avec cette algorithme. Concluez sur l'utilité (ou l'inutilité) de la phase d'exploration. Comparez les performances d'UCB avec celles de l'algorithme glouton.**

*[Ajoutez votre commentaire ici]*

**Q11. Testez différentes valeurs pour $C$ et trouvez sa valeur optimale expérimentalement.**

*[Ajoutez votre commentaire ici]*

# Echantillonnage de Thomson

Cet algorithme propose de modéliser la variable aléatoire de chaque vaccin avec une loi $\beta$ dont les paramètres $a$ et $b$ correspondent au nombre de patients que le vaccin a immunisés (resp. non immunisés).

Pour chaque patient, on tire un valeur aléatoire pour la loi $\beta$ décrivant chaque vaccin, puis on choisit le vaccin avec la plus grande valeur tirée. 

**Q12. Implémentez cet algorithme. Conservez les deux phases exploration/exploitation décrites ci-dessus. En prenant les valeurs de $N$ et $M$ trouvées à la question Q5, quel regret obtenez-vous ? Faites l'expérience avec au moins 10 MAB différents (tous ayant 5 vaccins) afin de calculer la moyenne et l'écart-type du regret.**

*[Ajoutez votre commentaire ici]*

**Q13. Reprenez la question Q4, mais cette fois-ci, vous representerez le taux d'immunisation empirique avec un [graphique en violon](https://en.wikipedia.org/wiki/Violin_plot) qui représente la loi beta associée à chaque vaccin.**

*[Ajoutez votre commentaire ici]*

**Q14. Représentez son regret pour différentes tailles du training set (comme dans la Q5). Comparez le regret avec les autres algorithmes.**

*[Ajoutez votre commentaire ici]*

# Conclusion

**Q15. Calculez le regret des algorithmes glouton, UCB & Thomson lorsqu'il y a un grand nombre de vaccins disponibles (K=100) (on prendra N=100). Faites le lien avec la [malédiction de la dimension](https://fr.wikipedia.org/wiki/Fl%C3%A9au_de_la_dimension).**

*[Ajoutez votre commentaire ici]*